In [0]:

import pandas as pd
import numpy as np

from importlib.machinery import SourceFileLoader
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.impute import SimpleImputer

DI = SourceFileLoader("DI", "/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/6.Source/src/DI.py").load_module()

Handler = DI.Handler.Handler()
Graph = DI.Graph.Graph()
AlgoML = DI.AlgoML.AlgoML()
Refine = DI.Refine.Refine()
Metrics = DI.Metrics.Metrics()

client_one = pd.read_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/1.Original/estaticos_portfolio1.csv")
client_two = pd.read_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/1.Original/estaticos_portfolio2.csv")
client_three = pd.read_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/1.Original/estaticos_portfolio3.csv")
market = pd.read_csv("/content/drive/My Drive/CodenationFinalProject/20200608_DS_codenationFinalProject/1.Original/estaticos_market.csv")

In [0]:
features_ramo = [
    "de_ramo",
    "setor",
    "nm_divisao",
    "nm_segmento",
]

features_burocracia = [
    "natureza_juridica_macro",
    "de_natureza_juridica",
    "fl_optante_simples"
]

features_consolidada = [
    "de_faixa_faturamento_estimado", 
    "de_faixa_faturamento_estimado_grupo",
    "idade_emp_cat"
]

features = features_ramo + features_burocracia + features_consolidada

x = market[features]
y = client_one[features]
# x.head(3)

In [0]:
# Obtendo coluna com no máximo n% de dados faltantes

p20_data = Refine.filter_columns_percent_null(market, 0.2)

In [34]:
# Lidar com os nulls de cada coluna de maneira que mais faça coerência para cada | CATEGORICA
# ver resultado em iloc[32:40] 
# SEM INFORMACAO é como vem padrao em de_faixa_faturamento_estimado, vamos usar disso para prencher então nossos null 

def imputerCategorical(df):
  df.loc[:, features] = SimpleImputer(fill_value = "SEM INFORMACAO", strategy='constant').fit_transform(df[features])


imputerCategorical(x)
imputerCategorical(y)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib

In [35]:
def oneEncoder(df):
  categorical_df = df.loc[:, ['de_ramo', 'setor', 'nm_divisao', 'nm_segmento',"natureza_juridica_macro",
                                        "de_natureza_juridica", "fl_optante_simples", 'de_faixa_faturamento_estimado',
                                        'de_faixa_faturamento_estimado_grupo', 'idade_emp_cat']].columns
  dummy = None
  start = False
  for categorical in list(categorical_df):
    if(not start):
      dummy = pd.get_dummies(df[categorical], columns=categorical, prefix=f"d_{categorical}_")
      start = True
    else:
      _dummy = pd.get_dummies(df[categorical], columns=categorical, prefix=f"d_{categorical}_")
      dummy = dummy.join(_dummy)

  return dummy

x_c = oneEncoder(x)
y_c = oneEncoder(y)

print(f"x_c_shape = {x_c.shape} \t y_c_shape = {y_c.shape}")
x_c

x_c_shape = (462298, 256) 	 y_c_shape = (555, 49)


,"d_de_ramo__ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",d_de_ramo__ATIVIDADES IMOBILIARIAS,d_de_ramo__BENS DE CONSUMO,d_de_ramo__COMERCIO E REPARACAO DE VEICULOS,d_de_ramo__COMERCIO POR ATACADO,d_de_ramo__COMERCIO VAREJISTA,"d_de_ramo__CULTURA, ESPORTE E RECREACAO",d_de_ramo__ELETRICIDADE E GAS,d_de_ramo__ELETROELETRONICOS,d_de_ramo__FARMACEUTICA,d_de_ramo__INDUSTRIA AUTOMOTIVA,d_de_ramo__INDUSTRIA DA CONSTRUCAO,d_de_ramo__INDUSTRIA DIGITAL,d_de_ramo__INDUSTRIAS EXTRATIVAS,d_de_ramo__ORGANISMOS INTERNACIONAIS,d_de_ramo__OUTROS,d_de_ramo__PAPEL E CELULOSE,d_de_ramo__PRODUTOS DE AGROPECUARIA,d_de_ramo__QUIMICA-PETROQUIMICA,"d_de_ramo__SEGUROS, PLANOS DE PREVIDENCIA E PLANOS DE SAUDE",d_de_ramo__SERVICOS ADMINISTRATIVOS,d_de_ramo__SERVICOS DE ALOJAMENTO/ALIMENTACAO,d_de_ramo__SERVICOS DE EDUCACAO,d_de_ramo__SERVICOS DE SANEAMENTO BASICO,d_de_ramo__SERVICOS DE SAUDE,d_de_ramo__SERVICOS DIVERSOS,d_de_ramo__SERVICOS FINANCEIROS,"d_de_ramo__SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",d_de_ramo__SERVICOS SOCIAIS,d_de_ramo__SIDERURGICA-METALURGIA,d_de_ramo__TELECOM,d_de_ramo__TEXTEIS,"d_de_ramo__TRANSPORTE, ARMAZENAGEM E CORREIO",d_setor__AGROPECUARIA,d_setor__COMERCIO,d_setor__CONSTRUÇÃO CIVIL,d_setor__INDUSTRIA,d_setor__SEM INFORMACAO,d_setor__SERVIÇO,d_nm_divisao__ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,...,d_de_natureza_juridica__SOCIEDADE EMPRESARIA LIMITADA,d_de_natureza_juridica__SOCIEDADE MERCANTIL DE CAPITAL E INDUSTRIA,d_de_natureza_juridica__SOCIEDADE SIMPLES EM COMANDITA SIMPLES,d_de_natureza_juridica__SOCIEDADE SIMPLES EM NOME COLETIVO,d_de_natureza_juridica__SOCIEDADE SIMPLES LIMITADA,d_de_natureza_juridica__SOCIEDADE SIMPLES PURA,d_de_natureza_juridica__SOCIEDADE UNIPESSOAL DE ADVOCACIA,d_fl_optante_simples__False,d_fl_optante_simples__True,d_fl_optante_simples__SEM INFORMACAO,d_de_faixa_faturamento_estimado__ACIMA DE 1 BILHAO DE REAIS,"d_de_faixa_faturamento_estimado__ATE R$ 81.000,00","d_de_faixa_faturamento_estimado__DE R$ 1.500.000,01 A R$ 4.800.000,00","d_de_faixa_faturamento_estimado__DE R$ 10.000.000,01 A R$ 30.000.000,00","d_de_faixa_faturamento_estimado__DE R$ 100.000.000,01 A R$ 300.000.000,00","d_de_faixa_faturamento_estimado__DE R$ 30.000.000,01 A R$ 100.000.000,00","d_de_faixa_faturamento_estimado__DE R$ 300.000.000,01 A R$ 500.000.000,00","d_de_faixa_faturamento_estimado__DE R$ 360.000,01 A R$ 1.500.000,00","d_de_faixa_faturamento_estimado__DE R$ 4.800.000,01 A R$ 10.000.000,00","d_de_faixa_faturamento_estimado__DE R$ 500.000.000,01 A 1 BILHAO DE REAIS","d_de_faixa_faturamento_estimado__DE R$ 81.000,01 A R$ 360.000,00",d_de_faixa_faturamento_estimado__SEM INFORMACAO,d_de_faixa_faturamento_estimado_grupo__ACIMA DE 1 BILHAO DE REAIS,"d_de_faixa_faturamento_estimado_grupo__ATE R$ 81.000,00","d_de_faixa_faturamento_estimado_grupo__DE R$ 1.500.000,01 A R$ 4.800.000,00","d_de_faixa_faturamento_estimado_grupo__DE R$ 10.000.000,01 A R$ 30.000.000,00","d_de_faixa_faturamento_estimado_grupo__DE R$ 100.000.000,01 A R$ 300.000.000,00","d_de_faixa_faturamento_estimado_grupo__DE R$ 30.000.000,01 A R$ 100.000.000,00","d_de_faixa_faturamento_estimado_grupo__DE R$ 300.000.000,01 A R$ 500.000.000,00","d_de_faixa_faturamento_estimado_grupo__DE R$ 360.000,01 A R$ 1.500.000,00","d_de_faixa_faturamento_estimado_grupo__DE R$ 4.800.000,01 A R$ 10.000.000,00","d_de_faixa_faturamento_estimado_grupo__DE R$ 500.000.000,01 A 1 BILHAO DE REAIS","d_de_faixa_faturamento_estimado_grupo__DE R$ 81.000,01 A R$ 360.000,00",d_de_faixa_faturamento_estimado_grupo__SEM INFORMACAO,d_idade_emp_cat__1 a 5,d_idade_emp_cat__10 a 15,d_idade_emp_cat__15 a 20,d_idade_emp_cat__5 a 10,d_idade_emp_cat__<= 1,d_idade_emp_cat__> 20
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,

In [0]:
# set(x_c.columns).intersection(set(y_c.columns))

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

coss = cosine_similarity(x_c[y_c.columns], [y_c.iloc[0]])

# a = pd.Series(coss[:,0]).sort_values(ascending=False)
# a[a == 1.0]

a = pd.Series(coss[:,0])
coss.shape

(462298, 1)

In [54]:
indexes = a[a == 1.0].index
recomended = x.iloc[indexes]

# recomended["nm_segmento"].unique()


# features_ramo = [
#     "de_ramo",
#     "setor",
#     "nm_divisao",
#     "nm_segmento",
# ]

# y

recomended
indexes

Int64Index([    45,     93,    108,    125,    369,    514,    770,    806,
               826,    853,
            ...
            461490, 461604, 461618, 461676, 461894, 461933, 461978, 462073,
            462098, 462239],
           dtype='int64', length=6098)

In [39]:
y['idade_emp_cat'].unique()

array(['<= 1', '5 a 10', '> 20', '15 a 20', '1 a 5', '10 a 15'],
      dtype=object)

In [40]:
print(y[y['idade_emp_cat'] == '<= 1'].shape)
print(y[y['idade_emp_cat'] == '5 a 10'].shape)
print(y[y['idade_emp_cat'] == '> 20'].shape)
print(y[y['idade_emp_cat'] == '15 a 20'].shape)
print(y[y['idade_emp_cat'] == '1 a 5'].shape)
print(y[y['idade_emp_cat'] == '10 a 15'].shape)

(56, 10)
(144, 10)
(92, 10)
(51, 10)
(157, 10)
(55, 10)
